### Import necessary Libraries

In [1]:
import json

import numpy as np
import pandas as pd

#### Assumptions

We set the base price and the seed for the random number generator.

In [2]:
base = 100

seed = 2021

#### Random number generator

In [3]:
rng = np.random.default_rng(seed)

### Upload the raffle data

In [4]:
data = json.loads('{"Address":{"0":"0x0123456789abcdef0000000000000000000001","1":"0x0123456789abcdef0000000000000000000002","2":"0x0123456789abcdef0000000000000000000003","3":"0x0123456789abcdef0000000000000000000004","4":"0x0123456789abcdef0000000000000000000005","5":"0x0123456789abcdef0000000000000000000006","6":"0x0123456789abcdef0000000000000000000007","7":"0x0123456789abcdef0000000000000000000008","8":"0x0123456789abcdef0000000000000000000009","9":"0x0123456789abcdef0000000000000000000010","10":"0x0123456789abcdef0000000000000000000011","11":"0x0123456789abcdef0000000000000000000012","12":"0x0123456789abcdef0000000000000000000013","13":"0x0123456789abcdef0000000000000000000014","14":"0x0123456789abcdef0000000000000000000015","15":"0x0123456789abcdef0000000000000000000016","16":"0x0123456789abcdef0000000000000000000017","17":"0x0123456789abcdef0000000000000000000018","18":"0x0123456789abcdef0000000000000000000019","19":"0x0123456789abcdef0000000000000000000020","20":"0x0123456789abcdef0000000000000000000021","21":"0x0123456789abcdef0000000000000000000022","22":"0x0123456789abcdef0000000000000000000023","23":"0x0123456789abcdef0000000000000000000024","24":"0x0123456789abcdef0000000000000000000025","25":"0x0123456789abcdef0000000000000000000026","26":"0x0123456789abcdef0000000000000000000027","27":"0x0123456789abcdef0000000000000000000028","28":"0x0123456789abcdef0000000000000000000029","29":"0x0123456789abcdef0000000000000000000030","30":"0x0123456789abcdef0000000000000000000031","31":"0x0123456789abcdef0000000000000000000032","32":"0x0123456789abcdef0000000000000000000033","33":"0x0123456789abcdef0000000000000000000034","34":"0x0123456789abcdef0000000000000000000035","35":"0x0123456789abcdef0000000000000000000036","36":"0x0123456789abcdef0000000000000000000037","37":"0x0123456789abcdef0000000000000000000038","38":"0x0123456789abcdef0000000000000000000039","39":"0x0123456789abcdef0000000000000000000040","40":"0x0123456789abcdef0000000000000000000041","41":"0x0123456789abcdef0000000000000000000042","42":"0x0123456789abcdef0000000000000000000043","43":"0x0123456789abcdef0000000000000000000044","44":"0x0123456789abcdef0000000000000000000045","45":"0x0123456789abcdef0000000000000000000046","46":"0x0123456789abcdef0000000000000000000047","47":"0x0123456789abcdef0000000000000000000048","48":"0x0123456789abcdef0000000000000000000049","49":"0x0123456789abcdef0000000000000000000050"},"Stake":{"0":161,"1":107,"2":136,"3":143,"4":225,"5":148,"6":188,"7":185,"8":100,"9":297,"10":174,"11":100,"12":155,"13":100,"14":190,"15":100,"16":107,"17":188,"18":186,"19":179,"20":182,"21":100,"22":165,"23":100,"24":100,"25":141,"26":148,"27":100,"28":100,"29":160,"30":100,"31":166,"32":118,"33":100,"34":128,"35":136,"36":250,"37":163,"38":100,"39":209,"40":100,"41":180,"42":100,"43":107,"44":178,"45":185,"46":100,"47":101,"48":100,"49":136},"Tokens":{"0":10,"1":12,"2":0,"3":7,"4":0,"5":3,"6":12,"7":7,"8":14,"9":14,"10":16,"11":0,"12":12,"13":0,"14":18,"15":18,"16":14,"17":6,"18":7,"19":2,"20":6,"21":12,"22":0,"23":8,"24":5,"25":0,"26":20,"27":0,"28":10,"29":12,"30":12,"31":4,"32":9,"33":0,"34":0,"35":2,"36":3,"37":1,"38":18,"39":0,"40":10,"41":4,"42":8,"43":8,"44":4,"45":1,"46":9,"47":8,"48":2,"49":0}}')
df = pd.DataFrame(data=data)
df.index = range(df.shape[0])

### Definition of the Ticket Weight Function

$$
\textrm{TWF} (\textrm{base}, \textrm{stake}, \textrm{tokens}) = \big( \textrm{stake} - \textrm{base} \big)^m \times \big(\textrm{token}+1\big)^n 
$$

In [5]:
m = 1
n = 2
def TWF(inputs):
    stake = inputs[0]
    token = inputs[1]
    return (stake-base) ** m * (token+1) ** n

### Calculate the Ticket Weight
We calculate the ticket weights of the participants using the TFT function defined above.

In [6]:
df['ticketWeight'] = df[['Stake','Tokens']].apply(TWF,axis=1)

### Round 1
Now we choose the first winner.

Using ticket weights, we calculated the probability of winning for each participant. Note that the probability is proportional to the ticket weight, i.e.,



$$
\mathcal{P} \{ \textrm{participant A wins the round 1} \} = \frac{\textrm{the ticket weight of the pariticipant A}}{\textrm{the sum of all weights of the participants in the Round 1}}
$$

In [7]:
probabilities = (df['ticketWeight'] / df['ticketWeight'].sum()).tolist()

##### Random Number
Now chood a random number between 0 and 1.

In [8]:
rdn = rng.random()

##### Find the winner
The winner is chosen in the following way:

Given the probabilities of the participants that sums to 1, each participants occupies part of the unit interval $[0,1]$. Then the first participant occupies from the 0 and the last participant does from 1- his/her probability to 1.

Now the winner is chosen if the random number belongs to the interval of the winner.

In [9]:
interval_points = [sum(probabilities[:n+1]) for n in range(len(probabilities))]

In [10]:
winnerIndex = len([x for x in interval_points if x < rdn])

##### Winner of Round 1

In [11]:
winnerAddress = df.loc[winnerIndex,'Address']
print('The winner of Round 1 has address {}.'.format(winnerAddress))
print('The winner had the winning probability {:5.1f}%.'.format(100*round(probabilities[winnerIndex],3)))

The winner of Round 1 has address 0x0123456789abcdef0000000000000000000019.
The winner had the winning probability   2.8%.


### Round 2
First eliminate the winner.

In [12]:
df_Round2 = df[df['Address']!=winnerAddress]
df_Round2 = df_Round2.reset_index(drop=True)

And calculate the probabilities of winning in Round 2.

In [13]:
probabilitiesRound2 = (df_Round2['ticketWeight'] / df_Round2['ticketWeight'].sum()).tolist()

##### Random number for Round 2

In [14]:
rdn = rng.random()

##### Find the winner of Round 2
The winner is chosen as before in the following way:

Given the probabilities of the participants that sums to 1, each participants occupies part of the unit interval $[0,1]$. Then the first participant occupies from the 0 and the last participant does from 1- his/her probability to 1.

Now the winner is chosen if the random number belongs to the interval of the winner.

In [15]:
interval_points = [sum(probabilitiesRound2[:n+1]) for n in range(len(probabilitiesRound2))]

In [16]:
winnerIndex = len([x for x in interval_points if x < rdn])

##### Winner of Round 2

In [17]:
winnerAddress = df_Round2.loc[winnerIndex,'Address']
print('The winner of Round 2 has address {}.'.format(winnerAddress))
print('The winner had the winning probability {:5.1f}%.'.format(100*round(probabilitiesRound2[winnerIndex],3)))

The winner of Round 2 has address 0x0123456789abcdef0000000000000000000032.
The winner had the winning probability   0.9%.
